In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib
import torch
import torch_geometric
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data

In [2]:
# Load data from csv files

dataset = 2
ef1 = np.loadtxt("dataset/dataset_"+str(dataset)+"/train/edge_features.csv.gz", dtype=float, delimiter=',')
e1 = np.loadtxt("dataset/dataset_"+str(dataset)+"/train/edges.csv.gz", dtype=int, delimiter=',')
gl1 = np.nan_to_num(np.loadtxt("dataset/dataset_"+str(dataset)+"/train/graph_labels.csv.gz", dtype=float, delimiter=','), nan=1)
nf1 = np.loadtxt("dataset/dataset_"+str(dataset)+"/train/node_features.csv.gz", dtype=float, delimiter=',')
num_nodes = np.loadtxt("dataset/dataset_"+str(dataset)+"/train/num_nodes.csv.gz", dtype=int, delimiter=',')
num_edges = np.loadtxt("dataset/dataset_"+str(dataset)+"/train/num_edges.csv.gz", dtype=int, delimiter=',')

In [3]:
i = 0
print(e1[np.sum(num_edges[:i]) : np.sum(num_edges[:i+1]), :].T)

[[0 1 1 2 1 3 3 4 4 5 5 6 6 7 6 8 8 9 9 3]
 [1 0 2 1 3 1 4 3 5 4 6 5 7 6 8 6 9 8 3 9]]


In [4]:
# Visualize graph

num_graphs = num_nodes.size
start = 125
graphs = []
data = []
for i in range(0, num_graphs):
    num_node = num_nodes[i]
    num_edge = num_edges[i]
    edges = e1[np.sum(num_edges[:i]) : np.sum(num_edges[:i+1]), :]
    edge_features = torch.FloatTensor(ef1[np.sum(num_edges[:i]) : np.sum(num_edges[:i+1]), :])
    node_features = torch.FloatTensor(nf1[np.sum(num_nodes[:i]) : np.sum(num_nodes[:i+1]), :])
    node_data = dict((j, node_features[j, :]) for j in range(num_node))
    # print(node_data[0])
    label = float(gl1[i])
    G = nx.Graph(y=label)
    G.add_nodes_from([i for i in range(num_node)])
    for j, e in enumerate(edges):
        G.add_edge(e[0], e[1], edge_attr=edge_features[j])
    nx.set_node_attributes(G, node_data, name="X")
    d = Data(edge_index=edges.T, x=node_data, edge_attr=edge_features)
    data.append(d)
    graphs.append(G)

# ---------- Code for drawing graph -----------

    # pos = nx.spring_layout(G)
    # nx.draw_networkx(G, pos=pos, with_labels=True)
    # nx.draw_networkx_edge_labels(G, pos=pos, font_size=5, clip_on=False)
    # print(label)
    # plt.show()
    # break

# print(gl1)

In [5]:
# Create a dataset
# for now using a list of data objects for dataset

# data = []
# for G in graphs:
#     # d = torch_geometric.utils.from_networkx(G)
#     d = Data(x = G)
#     # print(d)
#     # k = np.max(d.edge_index.numpy())
#     # if k > d.num_nodes-1:
#     #     print("----------------------------------------")
#     # print(k)
#     data.append(d)

size = len(data)

# consider first 80% as test data and next 20% as validation data
train_size = 0.8
X_train = data[:int(train_size*size)]
X_val = data[int(train_size*size):]

# Get dataloader

# batch size for mini-batch optimization

batch_size = 100

loader = DataLoader(X_train, batch_size=batch_size, shuffle=True)
print(loader)


In [6]:
from torch_geometric.nn import GCNConv
from torch_geometric.nn.pool import global_mean_pool
import torch.nn.functional as F

In [7]:
# Basic GCN implementation

if torch.cuda.is_available():
    device = torch.device('cuda')
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')


class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels, cached=True, bias=None)
        self.conv2 = GCNConv(hidden_channels, out_channels, cached=True, bias=None)
        self.edge_linear = torch.nn.Linear(3, 1)
        # self.fc = torch.nn.Linear(hidden_channels, out_channels)
        # self.batch = batch

    def forward(self, x, edge_index, edge_attr, batch, size):
        temp = self.edge_linear(edge_attr)
        x = self.conv1(x, edge_index, temp).relu()
        x = self.conv2(x, edge_index, temp).relu()
        x = global_mean_pool(x, batch, size)
        print(x)
        x = F.sigmoid(x)
        # x = x[root_mask, :]
        return x

In [8]:
model = GCN(9, 50, 1)
# model, data = model.to(device), data.to(device)
# model = model.to(device)

optimizer = torch.optim.Adam([
    dict(params=model.conv1.parameters(), weight_decay=0),
    dict(params=model.conv2.parameters(), weight_decay=0),
    dict(params=model.edge_linear.parameters(), weight_decay=0)
],lr=0.001)

In [15]:
# model.train()
print("hello")
loss_fun = torch.nn.BCELoss()
for i, batch in enumerate(loader):
    print(1)
    model.train()
    optimizer.zero_grad()
    # batch = next(iter(loader))
    batch.to(device)
    print(batch)
    print(np.max(batch.edge_index.numpy()))
    out = model(batch.X, batch.edge_index, batch.edge_attr, batch.batch, batch.y.shape[0])
    # print(out)
    # print(batch.y)
    loss = loss_fun(out.reshape(1,-1), batch.y.reshape(1,-1))
    loss.backward()
    optimizer.step()
    print(float(loss))


hello


KeyError: 4